In [1]:
!pip install cerebras-cloud-sdk --quiet
!pip install PyPDF2 --quiet
!pip install python-docx --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.5 MB/s eta 0:00:00


In [2]:
import os
import requests
import json
from cerebras.cloud.sdk import Cerebras

In [3]:
os.environ["CEREBRAS_API_KEY"] = "Your API Key here"
os.environ["DEEPSEEK_API_KEY"] = "Your API Key here"

client = Cerebras(api_key=os.environ["CEREBRAS_API_KEY"])

In [4]:
# LLM Query Functions

def query_gpt_oss(prompt, max_tokens=800):
    """GPT-OSS via Cerebras - used for synthesis"""
    response = client.chat.completions.create(
        model="gpt-oss-120b",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.7
    )
    return response.choices[0].message.content

def query_llama(prompt, max_tokens=500):
    response = client.chat.completions.create(
        model="llama-3.3-70b",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.7
    )
    return response.choices[0].message.content

def query_deepseek(prompt, max_tokens=500):
    url = "https://api.deepseek.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ['DEEPSEEK_API_KEY']}",
        "Content-Type": "application/json",
    }
    data = {
        "model": "deepseek-chat",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }
    resp = requests.post(url, headers=headers, json=data)
    return resp.json()["choices"][0]["message"]["content"]

def query_qwen(prompt, max_tokens=500):
    response = client.chat.completions.create(
        model="qwen-3-235b-a22b-instruct-2507",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.7
    )
    return response.choices[0].message.content

In [5]:
# Document Loading

def load_document(path):
    ext = path.split(".")[-1].lower()

    if ext == "txt":
        with open(path, "r", encoding="utf-8") as f:
            return f.read()

    elif ext == "pdf":
        import PyPDF2
        reader = PyPDF2.PdfReader(open(path, "rb"))
        return "\n".join(page.extract_text() for page in reader.pages)

    elif ext == "docx":
        import docx
        doc = docx.Document(path)
        return "\n".join([p.text for p in doc.paragraphs])

    else:
        raise ValueError(f"Unsupported file type: {ext}")

In [6]:
# Delphi-Style Iterative Critique System

def generate_initial_critiques(document_text, task_instruction):
    """ROUND 1: Each LLM provides independent critique"""

    prompt = f"""You are an expert critic evaluating a document.

--- DOCUMENT START ---
{document_text}
--- DOCUMENT END ---

Task: {task_instruction}

Provide a thorough critique covering:
1. Strengths of the document
2. Weaknesses and gaps
3. Specific areas for improvement
4. Overall assessment

Be specific and actionable in your feedback."""

    print("ROUND 1: Generating initial independent critiques...")

    critiques = {}

    # Get critique from each model
    try:
        print("  - Querying Llama...")
        critiques["llama"] = query_llama(prompt)
    except Exception as e:
        print(f"    Error with Llama: {e}")
        critiques["llama"] = None

    try:
        print("  - Querying DeepSeek...")
        critiques["deepseek"] = query_deepseek(prompt)
    except Exception as e:
        print(f"    Error with DeepSeek: {e}")
        critiques["deepseek"] = None

    try:
        print("  - Querying Qwen...")
        critiques["qwen"] = query_qwen(prompt)
    except Exception as e:
        print(f"    Error with Qwen: {e}")
        critiques["qwen"] = None

    return critiques

In [7]:
def generate_refined_critiques(document_text, task_instruction, round1_critiques):
    """ROUND 2: Each LLM sees others' critiques and refines their own (Delphi iteration)"""

    # Prepare summary of other critiques for each model
    print("\nROUND 2: Generating refined critiques after seeing others' feedback...")

    refined_critiques = {}

    # Filter valid critiques from round 1
    valid_critiques = {k: v for k, v in round1_critiques.items() if v is not None}

    for model in ["llama", "deepseek", "qwen"]:
        if model not in valid_critiques:
            refined_critiques[model] = None
            continue

        # Gather OTHER models' critiques for this model to review
        other_critiques = ""
        for other_model, critique in valid_critiques.items():
            if other_model != model:
                other_critiques += f"\n\n=== CRITIQUE FROM {other_model.upper()} ===\n{critique}"

        prompt = f"""You are an expert critic. You previously evaluated a document.

--- DOCUMENT START ---
{document_text}
--- DOCUMENT END ---

Task: {task_instruction}

YOUR INITIAL CRITIQUE:
{round1_critiques[model]}

OTHER EXPERTS' CRITIQUES:
{other_critiques}

Now that you've seen what other experts identified, please:
1. Refine your initial critique
2. Add any important points you may have missed
3. Address any contradictions or different perspectives
4. Strengthen your recommendations based on the collective insights
5. Keep what was valuable in your original critique

Provide your REFINED critique below:"""

        try:
            print(f"  - Refining {model.capitalize()}'s critique...")

            if model == "llama":
                refined_critiques[model] = query_llama(prompt, max_tokens=600)
            elif model == "deepseek":
                refined_critiques[model] = query_deepseek(prompt, max_tokens=600)
            elif model == "qwen":
                refined_critiques[model] = query_qwen(prompt, max_tokens=600)

        except Exception as e:
            print(f"    Error refining {model}: {e}")
            # Fall back to original critique
            refined_critiques[model] = round1_critiques[model]

    return refined_critiques

In [8]:
def synthesize_final_critique_gpt_oss(critiques_round1, critiques_round2):
    """Synthesize using GPT-OSS - shows evolution from Round 1 to Round 2"""

    valid_r1 = {k: v for k, v in critiques_round1.items() if v is not None}
    valid_r2 = {k: v for k, v in critiques_round2.items() if v is not None}

    if not valid_r2:
        return "Error: No valid refined critiques were generated."

    # Format critiques showing progression
    critique_text = "=== ROUND 1: INITIAL INDEPENDENT CRITIQUES ===\n"
    for model, critique in valid_r1.items():
        critique_text += f"\n--- {model.upper()} ---\n{critique}\n"

    critique_text += "\n\n=== ROUND 2: REFINED CRITIQUES (AFTER SEEING OTHERS) ===\n"
    for model, critique in valid_r2.items():
        critique_text += f"\n--- {model.upper()} ---\n{critique}\n"

    synthesis_prompt = f"""You are synthesizing a Delphi-style expert panel critique process.

The document was evaluated in TWO ROUNDS:
- Round 1: Independent critiques
- Round 2: Refined critiques after experts saw each other's feedback

{critique_text}

Your task:
Create a comprehensive FINAL CRITIQUE that:
1. Integrates insights from all experts across both rounds
2. Highlights consensus points (what all experts agreed on)
3. Notes areas where perspectives evolved between rounds
4. Resolves any remaining contradictions thoughtfully
5. Provides clear, prioritized, actionable recommendations
6. Shows how the iterative process strengthened the analysis

Structure your synthesis as:
- Executive Summary
- Consensus Findings (what all experts agreed on)
- Key Evolution from Round 1 to Round 2
- Critical Weaknesses Identified
- Priority Recommendations
- Conclusion

Be thorough, insightful, and actionable."""

    print("\nSYNTHESIS: Using GPT-OSS to create final unified critique...")

    try:
        final_critique = query_gpt_oss(synthesis_prompt, max_tokens=1200)
        return final_critique
    except Exception as e:
        print(f"Error during GPT-OSS synthesis: {e}")
        return "Error: Failed to synthesize final critique."

In [11]:
def delphi_critique_system(document_path, task_instruction):
    """Main Delphi-style iterative critique system"""

    print(f"\n{'='*60}")
    print("DELPHI-STYLE MULTI-LLM CRITIQUE SYSTEM")
    print(f"{'='*60}\n")

    # Step 1: Load document
    print(f"Loading document: {document_path}")
    try:
        document_text = load_document(document_path)
        print(f"  ✓ Loaded ({len(document_text)} characters)\n")
    except Exception as e:
        print(f"  ✗ Error loading document: {e}")
        return None

    # Step 2: Round 1 - Independent critiques
    critiques_round1 = generate_initial_critiques(document_text, task_instruction)

    # Step 3: Round 2 - Refined critiques after seeing others
    critiques_round2 = generate_refined_critiques(
        document_text,
        task_instruction,
        critiques_round1
    )

    # Step 4: Final synthesis using GPT-OSS
    final_critique = synthesize_final_critique_gpt_oss(
        critiques_round1,
        critiques_round2
    )

    print("\n" + "="*60)
    print("DELPHI PROCESS COMPLETE")
    print("="*60 + "\n")

    return {
        "round1_critiques": critiques_round1,
        "round2_critiques": critiques_round2,
        "final_critique": final_critique
    }

In [9]:
# Save functions

def save_results(result, output_file="critique_results", format="txt"):
    """Save results in txt, docx, or pdf format"""

    if format == "txt":
        _save_as_txt(result, f"{output_file}.txt")
    elif format == "docx":
        _save_as_docx(result, f"{output_file}.docx")
    elif format == "pdf":
        _save_as_pdf(result, f"{output_file}.pdf")
    else:
        print(f"Unknown format: {format}. Defaulting to txt.")
        _save_as_txt(result, f"{output_file}.txt")

def _save_as_txt(result, filename):
    """Save as plain text file"""
    with open(filename, "w", encoding="utf-8") as f:
        f.write("="*60 + "\n")
        f.write("DELPHI-STYLE MULTI-LLM CRITIQUE RESULTS\n")
        f.write("="*60 + "\n\n")

        # Round 1 critiques
        f.write("ROUND 1: INITIAL INDEPENDENT CRITIQUES\n")
        f.write("-"*60 + "\n\n")

        for model, critique in result["round1_critiques"].items():
            f.write(f"\n{'='*40}\n")
            f.write(f"{model.upper()} - ROUND 1\n")
            f.write(f"{'='*40}\n\n")
            f.write(critique if critique else "No critique generated\n")

        # Round 2 critiques
        f.write("\n\n" + "="*60 + "\n")
        f.write("ROUND 2: REFINED CRITIQUES (AFTER FEEDBACK)\n")
        f.write("-"*60 + "\n\n")

        for model, critique in result["round2_critiques"].items():
            f.write(f"\n{'='*40}\n")
            f.write(f"{model.upper()} - ROUND 2 (REFINED)\n")
            f.write(f"{'='*40}\n\n")
            f.write(critique if critique else "No refined critique\n")

        # Final synthesized critique
        f.write("\n\n" + "="*60 + "\n")
        f.write("FINAL SYNTHESIS (GPT-OSS)\n")
        f.write("="*60 + "\n\n")
        f.write(result["final_critique"])

    print(f"\n✓ Results saved to {filename}")

def _save_as_docx(result, filename):
    """Save as Word document with formatting"""
    from docx import Document
    from docx.shared import Pt, RGBColor
    from docx.enum.text import WD_ALIGN_PARAGRAPH

    doc = Document()

    # Title
    title = doc.add_heading("Delphi-Style Multi-LLM Critique Results", 0)
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER

    doc.add_paragraph()

    # Round 1
    doc.add_heading("Round 1: Initial Independent Critiques", 1)
    for model, critique in result["round1_critiques"].items():
        doc.add_heading(f"{model.upper()} - Round 1", 2)
        doc.add_paragraph(critique if critique else "No critique generated")
        doc.add_paragraph()

    doc.add_page_break()

    # Round 2
    doc.add_heading("Round 2: Refined Critiques (After Feedback)", 1)
    for model, critique in result["round2_critiques"].items():
        doc.add_heading(f"{model.upper()} - Round 2 (Refined)", 2)
        doc.add_paragraph(critique if critique else "No refined critique")
        doc.add_paragraph()

    doc.add_page_break()

    # Final
    doc.add_heading("Final Synthesis (GPT-OSS)", 1)
    doc.add_paragraph(result["final_critique"])

    doc.save(filename)
    print(f"\n✓ Results saved to {filename}")


In [12]:
# Main Implementation

if __name__ == "__main__":
    result = delphi_critique_system(
        document_path="/content/The Intelligence Age (Sep 2024).pdf",
        task_instruction="Critically evaluate this research document and identify weaknesses, gaps, and areas for improvement."
    )

    if result:
        # Print final critique
        print("\nFINAL CRITIQUE:")
        print("-"*60)
        print(result["final_critique"])

        save_results(result, "delphi_critique", format="docx")


DELPHI-STYLE MULTI-LLM CRITIQUE SYSTEM

Loading document: /content/The Intelligence Age (Sep 2024).pdf
  ✓ Loaded (6507 characters)

ROUND 1: Generating initial independent critiques...
  - Querying Llama...
  - Querying DeepSeek...
  - Querying Qwen...

ROUND 2: Generating refined critiques after seeing others' feedback...
  - Refining Llama's critique...
  - Refining Deepseek's critique...
  - Refining Qwen's critique...

SYNTHESIS: Using GPT-OSS to create final unified critique...

DELPHI PROCESS COMPLETE


FINAL CRITIQUE:
------------------------------------------------------------
**FINAL CRITIQUE – “The Intelligence Age” (September 23 2024)**  
*Synthesised from the Delphi‑style expert panel (LLAMA, DEEPSEEK, QWEN) after two rounds of independent and collaborative review.*

---

## 1. Executive Summary  

The manuscript offers an inspiring, historically‑anchored narrative that positions deep‑learning breakthroughs as the catalyst for a forthcoming “Intelligence Age.”  Its prose 